In [2]:
import sys
sys.path.append("../")

In [3]:
import os.path as op
import random

import matplotlib.pyplot as plt
import pandas as pd
import yaml

from agents import GreedyAgent
from agents.matchmanager import MatchManager
from agents.ml.classifier import ClassifierAgent
from agents.ml.regression import RegressionAgent
from core.const import RED, BLUE
from core.game.state import vectorStateInfo, vectorState, vectorActionInfo, vectorAction
from scenarios import scenarioJunction

%matplotlib inline
plt.rcParams['figure.figsize'] = (12,4)

In [5]:
with open(op.join('.', '../logger.config.yaml'), 'r') as stream:
    config = yaml.load(stream, Loader=yaml.FullLoader)
# logging.config.dictConfig(config)

In [6]:
def player(team:str, data:tuple, seed:int):
    if data[0] == 'gre':
        print(f'{team:5}-> GreedyAgent')
        return GreedyAgent(team, seed=seed)
    if data[0] == 'cls':
        print(f'{team:5}-> ClassifierAgent')
        return ClassifierAgent(team, data[1], seed=seed)
    if data[0] == 'reg':
        print(f'{team:5}-> RegressionAgent')
        return RegressionAgent(team, data[1], seed=seed)
    if data[0]=='ran':
        print(f'{team:5}-> RandomAgent')
        return RandomAgent(team, seed=seed)
        
    
    return None

In [7]:
def game(args):
    scenario,seed,agentRed,agentBlue=args

    board, state = scenario()
    
    print('seed:   ', seed)
    
    playerRed = player(RED, agentRed, seed)
    playerBlue = player(BLUE, agentBlue, seed)

    mm = MatchManager(' ', playerRed, playerBlue, board, state, seed=seed)
    while not mm.end:
        mm.nextStep()
    save={
        'scenario':board.name,
        'seed':42+i,
        'teamRed':playerRed.name,
        'teamBlue':playerBlue.name,
        'winner':mm.winner
    }
    file_name=f'../output/{playerRed.name}_{playerBlue.name}_{board.name}_{i}.json'
    #print(file_name)
    with open(file_name, 'w') as json_file:
        json.dump(save, json_file)
        #salva le cose su disco!
    
    print('winner: ', mm.winner)

    return mm

In [9]:
seed=51

In [10]:
red_models = [
    ('gre', ''),
    ('cls', 'models/Junction_RandomForestClassifier_red.joblib'),
    ('cls', 'models/Junction_RandomForestClassifier.joblib'),
    ('reg', 'models/Junction_RandomForestRegressor_red.joblib'),
    ('reg', 'models/Junction_RandomForestRegressor.joblib'),
    ('ran','')
]
blue_models = [
    ('gre', ''),
    ('cls', 'models/Junction_RandomForestClassifier_blue.joblib'),
    ('cls', 'models/Junction_RandomForestClassifier.joblib'),
    ('reg', 'models/Junction_RandomForestRegressor_blue.joblib'),
    ('reg', 'models/Junction_RandomForestRegressor.joblib'),
    ('ran','')

]

In [18]:
args=[]
scenarios = [scenarioJunction]

for i in range(0,100):
    for s in scenarios:
        s = random.randint(0, 1000000)
        r = random.randint(0, len(red_models)-1)
        b = random.randint(0, len(blue_models)-1)
        args.append((scenario,s,red_models[r],blue_models[b]))

In [ ]:
with Pool(64) as p:
    p.map(game, args)